In [1]:
import numpy as np 
import math 

L=4 
d_k=8 
d_v=8 

q =  np.random.randn(L,d_k)
k = np.random.randn(L,d_k)
v = np.random.randn(L,d_v)

In [2]:
print("Q\n",q )
print("K\n",k )
print("V\n",v )


Q
 [[-0.60087337  0.67126944  0.10155913 -0.56262346 -0.29803785 -0.64371564
  -1.77434951 -0.43247704]
 [ 1.41529116  1.15419432  0.94093452 -0.39003198 -0.33027558  0.31177575
   2.34971453 -1.24471776]
 [ 0.92799613  0.98204705  0.20427134  0.16590866  0.12791263  0.51208834
  -1.02985297 -0.8018781 ]
 [-1.18241538  1.69178656  1.35520237 -0.33319779 -0.33230956 -0.55309424
  -1.3147787   0.41832086]]
K
 [[ 0.03748189  1.32113663 -0.19559103  0.76798485 -0.61702103  0.68311583
   0.26122876 -1.70856663]
 [ 0.2688312  -0.58306262 -1.80113194 -0.21124517  0.01961988 -0.83930528
  -1.85188961 -0.44364818]
 [-0.61877628  0.0722166  -0.22168985 -0.35508975  1.45235031  0.18310505
  -0.83558277  0.63557983]
 [ 0.32635305  0.90611673  0.01438686  0.26540867  0.24560787  0.62358916
   0.15555017  0.65271815]]
V
 [[ 0.73787103  0.97619508 -0.62239097 -1.74958448  1.01466807 -0.20068913
   0.69767578  0.50861492]
 [-0.54120173 -2.68806698 -1.35214856 -0.72126805 -0.32167995 -0.89597551
  -0.1

#### Self attention

In [3]:
np.matmul(q,k.T)

array([[ 0.43193442,  3.39519801,  1.25456943, -0.76861295],
       [ 4.25158135, -5.97219815, -4.03958251,  1.08408892],
       [ 2.79158889,  1.1095493 ,  0.02291053,  0.90683094],
       [ 0.43883323, -0.96786673,  1.45228467,  0.7201449 ]])

In [4]:
# now divide by sqrt(d_k) since we want to reduce the variance 
q.var(), k.var(), np.matmul(q,k.T).var() 

(0.9050171287890647, 0.6214784005589036, 6.024063668856972)

In [5]:
scaled = np.matmul(q,k.T)/math.sqrt(d_k)
q.var(), k.var(), scaled.var() 


(0.9050171287890647, 0.6214784005589036, 0.7530079586071213)

In [6]:
# now we can see the reduction in variance of the producet 
scaled

array([[ 0.15271188,  1.20038377,  0.44355728, -0.27174571],
       [ 1.503161  , -2.1114909 , -1.42820809,  0.38328331],
       [ 0.98697572,  0.39228492,  0.0081001 ,  0.32061316],
       [ 0.15515098, -0.34219257,  0.51346017,  0.25460967]])

### Masking 
* This is to ensure words dont get context from words generated in future
* Not required in encoders(since all words are input at a time), but required in decoder 


In [8]:
mask = np.tril(np.ones((L,L)))
mask 

array([[1., 0., 0., 0.],
       [1., 1., 0., 0.],
       [1., 1., 1., 0.],
       [1., 1., 1., 1.]])

In [9]:
mask[mask==0] = -np.infty 
mask[mask==1] = 0

In [10]:
scaled + mask 

array([[ 0.15271188,        -inf,        -inf,        -inf],
       [ 1.503161  , -2.1114909 ,        -inf,        -inf],
       [ 0.98697572,  0.39228492,  0.0081001 ,        -inf],
       [ 0.15515098, -0.34219257,  0.51346017,  0.25460967]])

In [11]:
def softmax(x):
    return (np.exp(x).T / np.sum(np.exp(x),axis=-1)).T

In [12]:
attention = softmax(scaled + mask)

In [13]:
new_v = np.matmul(attention,v)
new_v

array([[ 0.73787103,  0.97619508, -0.62239097, -1.74958448,  1.01466807,
        -0.20068913,  0.69767578,  0.50861492],
       [ 0.70433339,  0.8801171 , -0.64152542, -1.72262173,  0.97962865,
        -0.21891974,  0.67520495,  0.46484521],
       [ 0.54907112, -0.2567708 , -0.66169692, -1.03503275,  0.51617155,
        -0.28533284,  0.71951006, -0.04196892],
       [ 0.48355077, -0.43529122, -0.35015941, -0.74901829,  0.47426155,
         0.30395061,  0.81219945,  0.1205363 ]])

In [14]:
v

array([[ 0.73787103,  0.97619508, -0.62239097, -1.74958448,  1.01466807,
        -0.20068913,  0.69767578,  0.50861492],
       [-0.54120173, -2.68806698, -1.35214856, -0.72126805, -0.32167995,
        -0.89597551, -0.15932599, -1.16069217],
       [ 1.64756231,  0.03189349,  0.24756256,  0.40598221,  0.41975721,
         0.38606912,  2.06811879,  0.13542511],
       [-0.69040216, -1.07804875, -0.32634916, -1.3546908 ,  0.49384491,
         1.31507778, -0.17619524,  0.45531578]])

In [19]:
def softmax(x):
    return (np.exp(x).T / np.sum(np.exp(x),axis=-1)).T

def scaled_dot_product_attention(q,k,v,mask=None):
    d_k = q.shape[-1]
    scaled = np.matmul(q,k.T)/math.sqrt(d_k)
    if mask is not None:
        scaled = scaled + mask 
    
    attention = softmax(scaled)
    out = np.matmul(attention,v)
    return out,attention 

In [23]:
 values, attention  = scaled_dot_product_attention(q,k,v,mask=mask)

In [24]:
print("Q\n",q)
print("K\n",k)
print("V\n",v)
print("New V \n", values )
print("Attention \n", attention)

Q
 [[-0.60087337  0.67126944  0.10155913 -0.56262346 -0.29803785 -0.64371564
  -1.77434951 -0.43247704]
 [ 1.41529116  1.15419432  0.94093452 -0.39003198 -0.33027558  0.31177575
   2.34971453 -1.24471776]
 [ 0.92799613  0.98204705  0.20427134  0.16590866  0.12791263  0.51208834
  -1.02985297 -0.8018781 ]
 [-1.18241538  1.69178656  1.35520237 -0.33319779 -0.33230956 -0.55309424
  -1.3147787   0.41832086]]
K
 [[ 0.03748189  1.32113663 -0.19559103  0.76798485 -0.61702103  0.68311583
   0.26122876 -1.70856663]
 [ 0.2688312  -0.58306262 -1.80113194 -0.21124517  0.01961988 -0.83930528
  -1.85188961 -0.44364818]
 [-0.61877628  0.0722166  -0.22168985 -0.35508975  1.45235031  0.18310505
  -0.83558277  0.63557983]
 [ 0.32635305  0.90611673  0.01438686  0.26540867  0.24560787  0.62358916
   0.15555017  0.65271815]]
V
 [[ 0.73787103  0.97619508 -0.62239097 -1.74958448  1.01466807 -0.20068913
   0.69767578  0.50861492]
 [-0.54120173 -2.68806698 -1.35214856 -0.72126805 -0.32167995 -0.89597551
  -0.1

### Multi-Head Attention

In [25]:
import numpy as np
import torch 
import torch.nn as nn
import torch.nn.functional as F

ModuleNotFoundError: No module named 'torch'

In [ ]:
!pip install torch 

  Obtaining dependency information for torch from https://files.pythonhosted.org/packages/ea/ea/4ab009e953bca6ff35ad75b8ab58c0923308636c182c145dc63084f7d136/torch-2.4.1-cp311-cp311-manylinux1_x86_64.whl.metadata
  Obtaining dependency information for typing-extensions>=4.8.0 from https://files.pythonhosted.org/packages/26/9f/ad63fc0248c5379346306f8668cda6e2e2e9c95e01216d2b8ffd9ff037d0/typing_extensions-4.12.2-py3-none-any.whl.metadata
  Obtaining dependency information for nvidia-cuda-nvrtc-cu12==12.1.105 from https://files.pythonhosted.org/packages/b6/9f/c64c03f49d6fbc56196664d05dba14e3a561038a81a638eeb47f4d4cfd48/nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata
  Obtaining dependency information for nvidia-cuda-runtime-cu12==12.1.105 from https://files.pythonhosted.org/packages/eb/d5/c68b1d2cdfcc59e72e8a5949a37ddb22ae6cade80cd4a57a84d4c8b55472/nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata
  Obtaining dependency information for nvidia

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00m eta 0:00:010:00:06m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 11.2 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 14.7 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 12.4 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 10.0 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.2/176.2 MB 8.2 MB/s eta 0:00:000m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.4/209.4 MB 5.2 MB/s eta 0:00:00m eta 0:00:010:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 6.5 MB/s eta 0:00:00m eta 0:00:010:00:01
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.7.1
    Uninstalling ty

In [ ]:
sequence_length =4 
batch_size=1 
input_dim = 512 
d_model = 512 
x = torch.randn((batch_size,sequence_length,input_dim))

In [ ]:
x.size()